# Search Engine Results

In [1]:
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import DanishStemmer
from textblob import TextBlob
import lemmy
from sklearn.metrics import pairwise_kernels
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sentence_transformers import SentenceTransformer, util
import tensorflow as tf
import torch

embedder = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

In [2]:
dataset_name = 'jobindex_cropped_bigger'

In [3]:
base = os.path.abspath('../')

def outname(name):
    return os.path.basename(dataset_name).split('.')[0] + name
    
outname_tfidf = outname('_distances_tfidf.csv')
outname_bert = outname('_encodings_bert.pt')
outname_df = outname('_preprocessed_df.csv')

print('Loading Processed Dataframe')
df = pd.read_csv(f'{base}/data/processed/{outname_df}', index_col=0)

print('Loading TFIDF distances')
%time tfidf = pd.read_csv( f'{base}/data/processed/{outname_tfidf}', index_col=0)

print('Loading BERT distances')
## It is not in Tensor format, but rather numpy array
# bert = pd.read_csv(f'{base}/data/processed/{outname_bert}', index_col=0)
%time bert = torch.load(f'{base}/data/processed/{outname_bert}')

print('\n All datasets loaded.')

Loading Processed Dataframe
Loading TFIDF distances
CPU times: user 1min 2s, sys: 2.74 s, total: 1min 5s
Wall time: 1min 7s
Loading BERT distances
CPU times: user 1.07 ms, sys: 27.5 ms, total: 28.6 ms
Wall time: 45.4 ms

 All datasets loaded.


In [4]:
tfidf = tfidf.reset_index()

In [5]:
## notice the corpus and title_processed
df.head(5)

,id,title,company,location,description,source,date,company_rating,type,description_img,company_rating_amount,ratings_link,link,date.1,merged,corpus,title_processed
0,2132,Østervang søger en social- og sundhedshjælper ...,Ikast-Brande Kommune,Klovborg,"Ikast-Brande Kommune, KlovborgStillingen er på...",NaN,2020-03-10,4 af 5 stjerner,paid,/img/brand/aeldre_jobindex_525_120.jpg,45.0,/virksomhed/26512/ikast-brande-kommune#show_ra...,http://www.ikast-brande.dk/,2020-03-10,Østervang søger en social- og sundhedshjælper ...,østervang søg social sundhedshjælp interes eng...,østervang søg social sundhedshjælp interes eng...
1,12247,Ørnereden i Lystrup-Elsted Dagtilbud søger en ...,Aarhus Kommune,Lystrup,DII Ørnereden søger en pædagog til en fast...,Jobcenter,2020-02-27,3 af 5 stjerner,unpaid,NaN,338.0,/virksomhed/8742/aarhus-kommune#show_rating,https://www.jobindex.dk/jobannonce/jobnet/8794964,2020-02-27,Ørnereden i Lystrup-Elsted Dagtilbud søger en ...,ørnered lystrup-elsted dagtilbud søg pædagog b...,ørnered lystrup-elsted dagtilbud søg pædagog b...
2,336,Ørepropper / Høreværn,NaN,SoundStoreXL.com A/S,Her finder du vores udvalg af ørepropper o...,SoundStoreXL.com A/S,2020-03-24,NaN,unpaid,NaN,NaN,NaN,https://www.soundstorexl.com/produkter/6089-oe...,2020-03-24,Ørepropper / Høreværn SoundStoreXL.com A/S 20...,øreprop høreværn soundstorexl.com a/s 2020-03-24,øreprop høreværn
3,8619,Ørelægen i Virum søger lægesekretær,Olin Øre Næse Hals Klinik v/Helle Birgitte Dah...,NaN,Olin Øre Næse Hals Klinik v/Helle Birgitte Dah...,NaN,2020-03-03,NaN,paid,/img/logo/Olin030320_logo.png,NaN,NaN,http://www.olinklinik.dk/,2020-03-03,Ørelægen i Virum søger lægesekretær Olin Øre N...,ørelæg virum søg lægesekretær olin øre næs hal...,ørelæg virum søg lægesekretær
4,6945,Øreklinikken dalgas plads søger sygeplejerske,Øreklinikken Dalgasplads,Herning,Øreklinikken Dalgas Plads søger sygeplejer...,Jobcenter,2020-03-05,NaN,unpaid,NaN,NaN,NaN,https://www.jobindex.dk/jobannonce/jobnet/8807347,2020-03-05,Øreklinikken dalgas plads søger sygeplejerske ...,øreklinik dalga plad søg sygeplejersk øreklini...,øreklinik dalga plad søg sygeplejersk


In [6]:
def preprocess_text(text):
    # text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = str(text).lower().strip()

    # caveat: this might conflict with the english text
    da_stop_words = stopwords.words('danish')
    stemmer = DanishStemmer()
    lemmatizer = lemmy.load("da")

    # remove plurals
    textblob = TextBlob(text)
    singles = [stemmer.stem(word) for word in textblob.words]

    # remove danish stopwords
    no_stop_words = [word for word in singles if word not in da_stop_words]

    # join text so it can be lemmatized
    joined_text = " ".join(no_stop_words)

    # lemmatization
    final_text = lemmatizer.lemmatize("", joined_text)

    return final_text[0]

## SEARCH QUERY

In [86]:
search_query = 'Pædagog psykiatrisk'
k = 20

## Direct custom search results

In [87]:
def find_direct_results(search_query):
    matching_entries = [df['title_processed'].index[df['title_processed'].str.contains(word, case=False)]
                        .values for word in search_query.split()]
    return list(set(matching_entries[0]).intersection(*matching_entries))

def print_direct_search_results(direct_results):
    agg_direct_results_indexes = []
    print('DIRECT RESULTS:', len(direct_results), '\n')
    for index, result in enumerate(direct_results):
        print(result, '-',  df['title'][result])
        agg_direct_results_indexes.append(result)
#     return agg_direct_results_indexes

direct_results = find_direct_results(preprocess_text(search_query))
print_direct_search_results(direct_results)

DIRECT RESULTS: 6 

3812 - Pædagog/SSA til Socialpsykiatrisk Center Skagen
1349 - Sygeplejerske/pædagog/SOSA til Ungdomspsykiatrisk Sengeafsnit 4 ved Børne- og Ungdomspsykiatrisk Afdeling, Herning
134 - pædagog, ergoterapeut, psykomototorisk terapeut med kompetencer indenfor psykiatri og misbrugsbehandling, i socialpsykiatrisk botilbud.
1548 - Sygeplejerske / Pædagog / SOSA til Ungdomspsykiatrisk Sengeafsnit 4 ved Børne- og Ungdomspsykiatrisk Afdeling, Herning
944 - To dygtige social- og sundhedsassistenter eller pædagoger til Socialpsykiatrisk Botilbud Rønnebo grundet opnormering
3839 - Pædagog til psykiatrisk afsnit O2 i Regionspsykiatrien Horsens


## TFIDF: Baseline model

In [88]:
%%time

def recommendations(data, results):
    sorted_results = []
    recommendation_indexes = []
    for result in results:
        sorted_distances = data[str(result)].sort_values().iteritems()
        
        for item_index, item_value in enumerate(sorted_distances):
            index, distance = item_value
            sorted_results.append({
                'item_index': index,
                'distance': distance,
                'title': df['title'][index]
            })
            
    agg_sorted_indexes = [x['item_index'] for x in sorted_results]
    ## removing direct_results for all results
    recommendation_indexes = [x for x in agg_sorted_indexes if x not in results]
        
    return recommendation_indexes

def display_results(data, name, direct_results):
    print (f'{name}\n')
    for index, recommendation in enumerate(recommendations(data=data, results=direct_results)[:k]):
        print(df['title'][recommendation])

def get_tfidf_recommendations(query):
    if len(direct_results) == 0:
        return None
    display_results(data=tfidf, name='TFIDF', direct_results=direct_results)

get_tfidf_recommendations(query=search_query)

TFIDF

Social- og sundhedsassistent i fortrinsvis dagvagt til Drachmannsvænget i Skagen
Social- og sundhedsassistent i fortrinsvis dagvagt til Drachmannsvænget i Skagen
Medarbejder med fokus på klimaudfordringer i Frederikshavn Kommune
Vil du skabe resultater i industri, produktion og lager Region Nordjylland
Socialrådgiver til Center for Familie
SSA eller SSH til Ravnshøj Hjemmepleje i fast stilling
SSA eller SSH til Ravnshøj Hjemmepleje i fast stilling
Medarbejder til døgnafdeling i Socialpsykiatrien i Vejen Kommune søges
Medarbejder til døgnafdeling i Socialpsykiatrien i Vejen Kommune søges
Coor søger en mødestabil Servicemedarbejder til rengøring hos vores kunde Falck i Skagen
Advokatsekretærelev
Butikschef til Skagen
Sygeplejerske i nattevagt til sengeafsnit N7 i Frederikshavn
Pædagoger
Sundhedsfaglig medarbejder til Socialpsykiatrisk Bosted
Pædagog til Dagtilbudsdistrikt Vest - Børnehuset Hånbæk
Pædagog til Dagtilbudsdistrikt Vest - Børnehuset Hånbæk
Pædagog til Dagtilbudsdistrik

# BERT

In [89]:
%%time 
embedded_query = embedder.encode(search_query, convert_to_tensor=True)
sims = util.pytorch_cos_sim(embedded_query, bert)[0]

results = sorted(range(len(sims)), key=lambda x: sims[x], reverse=True)[:k]
## removing direct_results for all results
filtered_results = [x for x in results if x not in direct_results]


print (f'BERT\n')
for recommendation in filtered_results:
    print(df['title'][recommendation]) 

BERT

Nattevagt i socialpsykiatrien
Engageret medarbejder til Socialpsykiatri & Rusmiddelsektionen
Engageret medarbejder til Socialpsykiatri & Rusmiddelsektionen
Psykolog til Rådgivning og Forebyggelse
Kollega søges til Ungeteamet Socialpsykiatrien
Sundhedsfaglig medarbejder til Socialpsykiatrisk Bosted
Sundhedsfaglig medarbejder til Socialpsykiatrisk Bosted
Relationelt stærk underviser med behandlingserfaring og indgående kendskab til unge med psykiatriske diagnoser
Sundhedskurser søger erfaren psykolog (genopslag)
Neurologisk fysioterapeut - barselsvikariat
Læge med erfaring indenfor misbrugsbehandling og psykiatri
Psykiatrisk fysioterapeut til tværgående rehabiliteringsteam på Sct. Hans
Socialfaglig koordinator søges til socialpsykiatrisk botilbud
Medarbejder til Socialpsykiatrien
Medarbejder til Socialpsykiatrien
Introduktion til psykiatri – et fantastisk spændende og udfordrende speciale
Introduktion til psykiatri – et fantastisk spændende og udfordrende speciale
'Rundt om psykiat